# **Import**

In [ ]:
import sys
sys.path.append('..')

from pydantic import ValidationError
from botocore.exceptions import ClientError
from app.async_storage import images_service, create_buckets
from app.schemas import UploadFileRequest, DownloadFileRequest, DeleteFileRequest, GetFileRequest

# **Initialize**

In [ ]:
await create_buckets()

In [ ]:
# Change directory to 'assets/' so that files can be referenced directly 
# (e.g., 'kitties.png' instead of 'assets/kitties.png')
%cd assets/
%ls

# **Test `FileService`**

**Note**: Both `documents_service` and `images_service` are instances of the same `FileService` class and share identical internal logic — differing only in their target bucket name. Therefore, it is sufficient to test the method on one service instance; successful behavior implies equivalent correctness for the other.

## **I. Test `upload()` method**

In [ ]:
async def test_upload_file(storage_key: str, file_path: str):
    try:
        request = UploadFileRequest(storage_key=storage_key, file_path=file_path)
        await images_service.upload(request)
        print(f"File '{storage_key}' uploaded successfully to bucket '{images_service.bucket_name}'.")
        return True
        
    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during upload ({error_code}): {e}")
        return False
        
    except Exception as e:
        print(f"Unexpected error during upload: {e}")
        return False

await test_upload_file(storage_key="kitties!", file_path="kitties.png")

In [ ]:
# Trying to upload a non-existent file
await test_upload_file(storage_key="kitties!", file_path="kitties")

## **II. Test `download()` method**

In [ ]:
async def test_download_file(storage_key: str, file_path: str):
    try:
        request = DownloadFileRequest(storage_key=storage_key, file_path=file_path)
        await images_service.download(request)
        print(f"File '{storage_key}' downloaded successfully to '{file_path}'.")
        return True
    
    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during download ({error_code}): {e}")
        return False
        
    except Exception as e:
        print(f"Unexpected error during download: {e}")
        return False

await test_download_file(storage_key="kitties!", file_path="kitties-from-minio!.png")

In [ ]:
# Trying to download a non-existent file
await test_download_file(storage_key="hello-there", file_path="kitties-from-minio!.png")

## **III. Test `get()` method**

In [ ]:
async def test_get_file_metadata(storage_key: str):
    try:
        request = GetFileRequest(storage_key=storage_key)
        metadata = await images_service.get(request)
        print(f"Metadata retrieved for '{storage_key}':")
        print(f"Size: {metadata['ContentLength']} bytes")
        print(f"LastModified: {metadata['LastModified']}")
        print(f"ETag: {metadata['ETag']}")
        return metadata

    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False
        
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during retrieval ({error_code}): {e}")
        return False
        
    except Exception as e:
        print(f"Unexpected error during retrieval: {e}")
        return False

await test_get_file_metadata(storage_key="kitties!")

## **IV. Test `get_all()` method**

In [ ]:
# Create a helper record to ensure get_all returns all files, not just one
await test_upload_file(storage_key="more-kitties!", file_path="more-kitties.png")

In [ ]:
async def test_list_files(max_keys: int = 10):
    try:
        objects = await images_service.get_all(max_keys=max_keys)
        if objects:
            print(f"Found {len(objects)} objects in bucket '{images_service.bucket_name}':")
        else:
            print(f"Bucket '{images_service.bucket_name}' is empty.")
        return objects
    
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during retrieval ({error_code}): {e}")
        return []
        
    except Exception as e:
        print(f"Unexpected error during retrieval: {e}")
        return []

await test_list_files()

## **V. Test `delete()` method**

In [ ]:
async def test_delete_file(storage_key: str):
    try:
        request = DeleteFileRequest(storage_key=storage_key)
        await images_service.delete(request)
        print(f"File '{storage_key}' deleted (or did not exist) from bucket '{images_service.bucket_name}'.")
        return True
    
    except ValidationError as e:
        print(f"Validation error (Pydantic): {e}")
        return False

    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"S3 ClientError during delete ({error_code}): {e}")
        return False
    
    except Exception as e:
        print(f"Unexpected error during delete: {e}")
        return False

await test_delete_file(storage_key="kitties!")

In [ ]:
# Trying to delete a non-existent file
await test_delete_file(storage_key="hello-there")

In [ ]:
# Trying to delete file with invalid name
await test_delete_file(storage_key=" dthrjjf   ")